In [3]:
import pandas as pd
import numpy as np

# Model 4.0

In [167]:
# YEARS

df_2011 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2011.xlsx")
df_2012 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2012.xlsx")
df_2013 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2013.xlsx")
df_2014 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2014.xlsx")
df_2015 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2015.xlsx")
df_2016 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2016.xlsx")
df_2017 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2017.xlsx")
df_2018 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2018.xlsx")
df_2019 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2019.xlsx")
df_2020 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2020.xlsx")
df_2021 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2021.xlsx")
df_2022 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2022.xlsx")
df_2023 = pd.read_excel("Data/Burglaries_coords/Burglaries locations 2023.xlsx")

In [36]:
#Importing POI dataframe
df_POI_new = pd.read_csv("Data/granular_pois.csv")


In [179]:
# This function is suppose to Provide a dictionary who's keys are the LSOA code, and the values of it are dictionaries,
# Dictionaries in the values have the poi categories as keys, and as values they have intgers representing the amount of the 
# poi in the LSOA code
def POI_per_LSOA(df_POI,map_barnet):
    
    Book_keeping_poi = {}
    
    for index, row in df_POI.iterrows():
        
        Poi = row[0]
        Longitude = row[1]
        Latitude = row[2] 
        
        LSOA_of_Poi = determine_LSOA(Longitude,Latitude, map_barnet)
        #print(Book_keeping_poi)
        if LSOA_of_Poi != False:
            
            if LSOA_of_Poi in Book_keeping_poi:
            
                Value = Book_keeping_poi[LSOA_of_Poi]
            
                if Poi in Value:
                    #print(Value)
                    Value[Poi] = Value[Poi] + 1
            
                else:
                
                    Value[Poi] = 1
                   
            else:
            
                Book_keeping_poi[LSOA_of_Poi] = {}
                Value = Book_keeping_poi[LSOA_of_Poi]
                
                if Poi in Value:
                    
                    Value[Poi] = Value[Poi] + 1
            
                else:
                
                    Value[Poi] = 1
            
    
    return Book_keeping_poi

In [71]:
def create_dataframe(data):
    
    columns = set()
    for d in data.values():
        columns.update(d.keys())
    columns.add('lsoa')  
    
    
    df = pd.DataFrame(columns=columns)
    
    
    for lsoa, values in data.items():
        
        row = {col: 0 for col in columns}
        row['LSOA'] = lsoa  
        
        row.update(values)
        
        df = df.append(row, ignore_index=True)
        
        column = df.pop('LSOA')
        df.insert(0, 'LSOA', column)
    
    return df

In [183]:
Dictionary_POI_per_LSOA = POI_per_LSOA(df_POI_new,map_barnet)
df_POI_per_LSOA = create_dataframe(D)

C:\Users\ninob\anaconda3\lib\site-packages\shapely\predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


In [138]:
#Lastly this function adds the fractions as well , that we wanted
def calculate_fractions(df):
    new_df = df.copy()  
    
    
    for column in df.columns:
        
        new_df[column] = pd.to_numeric(new_df[column], errors='coerce')
        
        
        fraction_col = round(new_df[column] / new_df.sum(axis=1), 2)
        
        
        new_df[column + '_fraction'] = fraction_col
    
   
    new_df = new_df.iloc[:,1:]
    new_df.insert(0, 'LSOA', df['LSOA'])
    new_df = new_df.dropna(axis=1, how='all')
    return new_df

In [185]:
df_POI_per_LSOA_with_fractions = calculate_fractions(df_POI_per_LSOA)
df_POI_per_LSOA_with_fractions

,LSOA,market,homeless shelter,gym,museum,church,park,bank,police,cafe,...,hospital_fraction,lsoa_fraction,hotel_fraction,school_fraction,soccer field_fraction,groceries_fraction,university_fraction,post_fraction,bus_fraction,bar_fraction
0,E01000188,0,0,0,0,0,2,1,0,1,...,0.00,0.0,0.08,0.15,0.0,0.07,0.00,0.00,0.37,0.00
1,E01000194,1,0,0,0,4,0,1,0,3,...,0.00,0.0,0.00,0.06,0.0,0.00,0.00,0.00,0.25,0.03
2,E01000193,2,0,1,0,1,0,3,0,6,...,0.00,0.0,0.03,0.06,0.0,0.09,0.00,0.03,0.18,0.00
3,E01000192,2,0,0,0,0,0,6,0,5,...,0.00,0.0,0.02,0.06,0.0,0.08,0.00,0.04,0.18,0.00
4,E01000125,0,0,1,0,2,1,0,0,0,...,0.14,0.0,0.00,0.20,0.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,E01000118,1,0,0,0,1,1,0,0,0,...,0.00,0.0,0.00,0.07,0.0,0.00,0.00,0.00,0.59,0.06
203,E01000119,0,0,0,0,2,0,0,0,0,...,0.00,0.0,0.00,0.31,0.0,0.00,0.00,0.00,0.15,0.00
204,E01000121,0,0,0,0,0,2,0,0,0,...,0.00,0.0,0.00,0.22,0.0,0.00,0.00,0.00,0.21,0.00
205,E01000115,1,0,0,0,0,0,0,0,0,...,0.00,0.0,0.19,0.37,0.0,0.00,0.00,0.00,0.17,0.00


### That was all about the POI per LSOA dataframe, Below you have model 4

In [186]:
def Model_4(df_burglaries, df_POI_per_LSOA ,map_barnet):
    
    df_POI_per_LSOA = df_POI_per_LSOA.iloc[:,:25]
    
    Book_keeping_burglaries = {}
    
    for index, row in df_burglaries.iterrows():
        
        Longitude = row[0]
        Latitude = row[1]
        
        LSOA_of_burglary = determine_LSOA(Longitude,Latitude, map_barnet)
        
        if LSOA_of_burglary in Book_keeping_burglaries:
            
            Book_keeping_burglaries[LSOA_of_burglary] = Book_keeping_burglaries[LSOA_of_burglary] + 1
        
        else:
            
            Book_keeping_burglaries[LSOA_of_burglary] = 0
            
    new_column_values = []


    for entry in df_POI_per_LSOA['LSOA']:
    
        if entry in Book_keeping_burglaries:
        
            new_column_values.append(Book_keeping_burglaries[entry])
        else:
        
            new_column_values.append(0)


    df_POI_per_LSOA['Amount of burglaries'] = new_column_values
    
    
    
    
    return df_POI_per_LSOA

### I take as example dataframe of the locations of burglaries in 2012

In [176]:
df = Model_4(df_2012, new_df ,map_barnet)
df